In [1]:
# !pip install -q --upgrade transformers
# !pip install -q --upgrade peft
!pip install -q -U "transformers==4.40.0" "peft==0.10.0"
!pip install -q --upgrade datasets
!pip install -q --upgrade accelerate
!pip install -q --upgrade bitsandbytes
!pip install -q --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, Value
from peft import PeftModel, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score
import numpy as np

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
test_df = pd.read_csv('test.csv')

In [5]:
train_df['label'] = train_df['label'].astype('category')
train_df['target'] = train_df['label'].cat.codes
val_df['label'] = val_df['label'].astype('category')
val_df['target'] = val_df['label'].cat.codes
test_df['label'] = test_df['label'].astype('category')
test_df['target'] = test_df['label'].cat.codes

In [63]:
# val_df.rename(columns={'target': 'label', 'label':'temp'}, inplace=True)
# val_df.rename(columns={'temp': 'target'}, inplace=True)
# train_df.rename(columns={'target': 'label', 'label':'temp'}, inplace=True)
# train_df.rename(columns={'temp': 'target'}, inplace=True)

In [6]:
category_codes = dict(enumerate(train_df['label'].cat.categories))
category_names = {v: k for k, v in category_codes.items()}

In [7]:
len(category_codes)

6

## Loop through dataset to measure performance before training/fitting the model

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# import pandas as pd
# import torch

In [ ]:
# model_name = "Qwen/Qwen2.5-0.5B"
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=len(category_codes),
#     trust_remote_code=True,
#     label2id = category_names,
#     id2label = category_codes
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = tokenizer.pad_token_id
# model.config.use_cache = False
# model.config.pretraining_tp = 1

In [ ]:
# samples_per_class = 6

# # Sample 10 rows for each target class (0 to 5)
# sampled_df = test_df.groupby('target').sample(n=samples_per_class, random_state=42)

# # Optional: reset index
# sampled_df = sampled_df.reset_index(drop=True)
# print(sampled_df)

# # Convert summaries to a list
# sentences = sampled_df.text.tolist()

In [ ]:
# # Define the batch size
# batch_size = 10  # You can adjust this based on your system's memory capacity

# # Initialize an empty list to store the model outputs
# all_outputs = []

# # Process the sentences in batches
# for i in range(0, len(sentences), batch_size):
#     # Get the batch of sentences
#     batch_sentences = sentences[i:i + batch_size]

#     # Tokenize the batch
#     inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

#     # Move tensors to the device where the model is (e.g., GPU or CPU)
#     inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

#     # Perform inference and store the logits
#     with torch.no_grad():
#         outputs = model(**inputs)
#         all_outputs.append(outputs['logits'])

# final_outputs = torch.cat(all_outputs)
# print(final_outputs.argmax(dim=1))
# sampled_df['predictions']=final_outputs.argmax(axis=1).numpy()
# sampled_df['predictions']=sampled_df['predictions'].map(category_codes)

# Convert from Pandas DataFrame to Hugging Face Dataset

In [8]:
# just for direct convert csv to dataset
# dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})
# or
dataset_train = Dataset.from_pandas(train_df.drop('label', axis=1))
dataset_val = Dataset.from_pandas(val_df.drop('label', axis=1))
dataset_test = Dataset.from_pandas(test_df.drop('label', axis=1))
dataset = DatasetDict({'train': dataset_train, 'val': dataset_val, 'test':dataset_test})

In [9]:
class_weights=(1/train_df.target.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights

tensor([0.1532, 0.1716, 0.1018, 0.0839, 0.1231, 0.3663])

# Load QWEN model

In [10]:
model_name = "Qwen/Qwen2.5-0.5B"

In [11]:
lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS,
                r=16,
                lora_alpha=32,
                target_modules= ["q_proj", "k_proj", "v_proj", "o_proj"],
                lora_dropout=0.05,
                bias="none"
            )

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(category_codes),
    trust_remote_code=True,
    label2id = category_names,
    id2label = category_codes
)
# model = prepare_model_for_kbit_training(model)

In [13]:
model = get_peft_model(model, lora_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [15]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

# Create Qwen tokenized

In [ ]:
def qwen_preprocess(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=512
    )
# # # profesional way to encode and tokenized dataset
# # encoded_dataset = {
# #     split: dataset[split].map(qwen_preprocess, batched=True)
# #     for split in ["train", "test"]
# # }
# # for split in encoded_dataset:
# #     encoded_dataset[split] = encoded_dataset[split].remove_columns(
# #         [col for col in encoded_dataset[split].column_names if col not in ["input_ids", "attention_mask", 'target']]
# #     )
# #     encoded_dataset[split].set_format(type="torch")

# simple way to encode and tokenized dataset
col_to_delete = ['text']
tokenized_datasets = dataset.map(qwen_preprocess, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format('torch')

In [17]:
print("Sample from tokenized_datasets['train'][0]:")
print(tokenized_datasets['train'][0])

print("\nFeatures of tokenized_datasets['train']:")
print(tokenized_datasets['train'].features)

Sample from tokenized_datasets['train'][0]:
{'label': tensor(4), 'input_ids': tensor([  220,   576, 57986,   315, 20486,  3100,   311,  4925,   646, 42744,
         1186, 77690, 55787,  2524,   916,  3684,  5888,    11,   448,  3387,
         2673, 35472, 59601,   323, 32197,  2142,  6839,   311,   387,   683,
        68755,   389, 23998,  9349,  4996,    83,   960,  1297,   882, 28405,
           13,  5692,    11,   553, 50197, 20486,  3100, 30606,   315,  2176,
        13482,   323, 26931, 65457,    11,   582,  1473,   429,  6757,   315,
         5189, 12616,   323, 25055, 23270,   646,   387,  3465,   448, 16052,
         6814,   279,  1156, 66644,   283,   258, 10143,    13,  5633, 24989,
        65457, 48753,   304,   264, 14071,  1616,  2176,   993,    72,   370,
          774, 10582,   370,   437, 11379,   438,  1632,   438, 12140,   946,
         7053,  3438,  7556,  1948,  1044,   763,   323,   390, 23113, 20892,
           11,   323,  1373,  1172,   264,   342,  5677, 12616, 

# Data Collator

In [18]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

# metrics to compute for evaluaition

In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}

# Define custom trainer with classweights

In [20]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        # labels = inputs.pop('label').long()
        # print("INPUT KEYS:", inputs.keys())
        label = inputs['labels'].long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.logits

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, label, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, label)

        return (loss, outputs) if return_outputs else loss

# define training args

In [21]:
training_args = TrainingArguments(
    output_dir="lora-results",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

<ipython-input-20-6f7c35c60e73>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


# Run trainer

In [23]:
train_result = trainer.train()

Epoch,Training Loss,Validation Loss,Balanced Accuracy,Accuracy
1,1.770200,0.984146,0.599689,0.702222
2,0.898400,0.774305,0.696576,0.777778


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#Check results

In [26]:
def get_performance_metrics(df_test):
  y_test = df_test.label
  y_pred = df_test.predictions

  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
  print("Accuracy Score:", accuracy_score(y_test, y_pred))

In [25]:
def make_predictions(model,df_test):


  # Convert summaries to a list
  sentences = test_df.text.tolist()

  # Define the batch size
  batch_size = 32  # You can adjust this based on your system's memory capacity

  # Initialize an empty list to store the model outputs
  all_outputs = []

  # Process the sentences in batches
  for i in range(0, len(sentences), batch_size):
      # Get the batch of sentences
      batch_sentences = sentences[i:i + batch_size]

      # Tokenize the batch
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_codes[l])


make_predictions(model,test_df)

In [28]:
get_performance_metrics(test_df)

Confusion Matrix:
[[15  0  1  0  0  0]
 [ 1 11  0  0  2  0]
 [ 0  0 21  0  1  0]
 [ 0  1  1 25  0  0]
 [ 1  5  3  0  6  0]
 [ 0  0  4  0  2  0]]

Classification Report:
                                            precision    recall  f1-score   support

                              astrophysics       0.88      0.94      0.91        16
electrical engineering and systems science       0.65      0.79      0.71        14
                high energy physics theory       0.70      0.95      0.81        22
                               mathematics       1.00      0.93      0.96        27
                                   physics       0.55      0.40      0.46        15
                           quantum physics       0.00      0.00      0.00         6

                                  accuracy                           0.78       100
                                 macro avg       0.63      0.67      0.64       100
                              weighted avg       0.74      0.78      0.75

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
metrics = train_result.metrics
max_train_samples = len(dataset_train)
metrics["train_samples"] = min(max_train_samples, len(dataset_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

***** train metrics *****
  epoch                    =        2.0
  total_flos               =  1410681GF
  train_loss               =     1.1687
  train_runtime            = 0:07:44.11
  train_samples            =       1350
  train_samples_per_second =      5.818
  train_steps_per_second   =      2.909


In [ ]:
trainer.save_model("saved_model")

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
%cd /content/saved_model
!zip -r saved_model.zip /content/saved_model
files.download('saved_model.zip')

In [ ]:
%cd /content/sample_data
!zip -r sample_data.zip /content/sample_data
files.download('sample_data.zip')

In [ ]:
%cd /content/lora-results
!zip -r lora-results.zip /content/lora-results
files.download('lora-results.zip')